# Gun Violence Dataset

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# gun violence data 2013-2018
df_gun_violence = pd.read_csv('/Users/Kenny/Desktop/DataSets/gun-violence-data_01-2013_03-2018.csv')
# df_gun_violence = pd.read_csv('https://media.githubusercontent.com/media/Cng000/Gun-violence-analysis/master/dataset/gun-violence-data_01-2013_03-2018.csv')

In [ ]:
# mean annual household income (2016)
# df_income = pd.read_csv('./dataset/2016_mean_income_raw_data.csv', skiprows=2)
df_income = pd.read_csv('https://raw.githubusercontent.com/Cng000/Gun-violence-analysis/master/dataset/2016_mean_income_raw_data.csv', skiprows=2)

In [ ]:
# population estimates
#df_state_populations = pd.read_csv('./dataset/population_estimates.csv', skiprows=1)
df_state_populations = pd.read_csv('https://raw.githubusercontent.com/Cng000/Gun-violence-analysis/master/dataset/population_estimates.csv', skiprows=1)

In [ ]:
# NRA spending
df_nra_spending = pd.read_csv('https://raw.githubusercontent.com/Cng000/Gun-violence-analysis/master/dataset/nra_spending_115th_congress.csv', skiprows=1)
# US state codes (source, https://github.com/jasonong/List-of-US-States/blob/master/states.csv)
df_state_codes = pd.read_csv('https://raw.githubusercontent.com/jasonong/List-of-US-States/master/states.csv')

In [ ]:
# unemployment data
# raw data: https://factfinder.census.gov/faces/tableservices/jsf/pages/productview.xhtml?pid=ACS_17_5YR_S2301&prodType=table
df_unemployment = pd.read_csv('./dataset/ACS_17_5YR_S2301_with_ann.csv', skiprows=1)


### Gun Violence Analysis

In [ ]:
df_gun_violence.info()

In [ ]:
df_gun_violence.head()

In [ ]:
# let's drop some columns
df_gun_violence.drop(['incident_id', 'address', 'incident_url', 'source_url', 'incident_url_fields_missing', 'latitude', 'location_description', 'longitude', 'notes', 'participant_name', 'participant_relationship', 'sources'], axis=1)

In [ ]:
# extract year from date
df_gun_violence['year'] = pd.to_numeric(df_gun_violence['date'].str[0:4], downcast='integer')

#### Top 10 cities/counties by number of incidents

In [ ]:
df_gun_violence['city_or_county'].value_counts().head(10)

#### Top 10 cities/counties by number of deaths

In [ ]:
df_gun_violence.groupby('city_or_county')['n_killed'].sum().sort_values(ascending=False).head(10)

#### Top 10 cities/counties by number of injured

In [ ]:
df_gun_violence.groupby('city_or_county')['n_injured'].sum().sort_values(ascending=False).head(10)

#### Top 10 states with higher number of incidents

In [ ]:
df_gun_violence['state'].value_counts().head(10)

#### Top 10 states with higher number of deaths

In [ ]:
df_gun_violence.groupby('state')['n_killed'].sum().sort_values(ascending=False).head(10)

#### Top 10 states with higher number of injuries

In [ ]:
df_gun_violence.groupby('state')['n_injured'].sum().sort_values(ascending=False).head(10)

#### Can we correlate state population to number of violent incidents/deaths?

In [ ]:
# first, let's look at gun incidents for year 2013 only
df_gun_violence_2013 = df_gun_violence.loc[df_gun_violence.year == 2013]['state'].value_counts().reset_index().rename(index=str, columns={'index':'state', 'state':'incident_counts_2013'})
df_gun_violence_2013

In [ ]:
# no data for 15 states in 2013. Also, incident counts appear to be too low for 2013
# Let's get the population estimate of each state now

In [ ]:
# extract 2013 population estimates
df_state_populations_2013 = df_state_populations[['Geography', 'Population Estimate (as of July 1) - 2013']]

df_state_populations_2013 = df_state_populations_2013.rename(index=str, columns={'Geography':'state', 'Population Estimate (as of July 1) - 2013':'population_estimate_2013'})
# merge number of incidents to population estimate (2013 only)
df_incidents_population_2013 = df_gun_violence_2013.merge(df_state_populations_2013, how='inner', on='state')
# plot results
df_incidents_population_2013.plot.scatter('population_estimate_2013', 'incident_counts_2013')

In [ ]:
# now, let's combine incident counts for all years. We settle on population estimate for the year 2017
df_incident_count_2013_2018 = df_gun_violence['state'].value_counts().reset_index().rename(index=str, columns={'index':'state', 'state':'incident_count_2013_2018'})
df_state_populations_2017 = df_state_populations[['Geography','Population Estimate (as of July 1) - 2017']].rename(index=str, columns={'Geography':'state', 'Population Estimate (as of July 1) - 2017':'population_estimate_2017'})
df_incident_count_population_estimate_2017 = df_incident_count_2013_2018.merge(df_state_populations_2017, how='inner', left_on='state', right_on='state')
df_incident_count_population_estimate_2017.plot.scatter('population_estimate_2017', 'incident_count_2013_2018')

In [ ]:
df_incident_count_population_estimate_2017.corr(method='pearson')

#### Mass Shootings

In [ ]:
# top ten states with most mass shootings from 2013 to 2018
df_gun_violence['mass_shooting'] = pd.to_numeric(df_gun_violence['incident_characteristics'].str.contains('Mass Shooting', case=False), downcast='integer')
df_gun_violence.loc[df_gun_violence['mass_shooting'] == 1.0]['state'].value_counts().head(10)

In [ ]:
# population estimate vs. mass shooting count
df_gun_violence.loc[df_gun_violence['mass_shooting'] == 1.0]['state'].value_counts().reset_index().rename(index=str, columns={'index':'state', 'state':'mass_shooting_count'}).merge(df_state_populations_2017, how='inner', left_on='state', right_on='state').plot.scatter('population_estimate_2017', 'mass_shooting_count')

#### Home Invasion

In [ ]:
# top ten states with most home invasions from 2013 to 2018
df_gun_violence['home_invasion'] = pd.to_numeric(df_gun_violence['incident_characteristics'].str.contains('Home Invasion', case=False), downcast='integer')
df_gun_violence.loc[df_gun_violence['home_invasion'] == 1.0]['state'].value_counts().head(10)

In [ ]:
# population estimate vs. home invasion count
df_gun_violence.loc[df_gun_violence['home_invasion'] == 1.0]['state'].value_counts().reset_index().rename(index=str, columns={'index':'state', 'state':'home_invasion_count'}).merge(df_state_populations_2017, how='inner', left_on='state', right_on='state').plot.scatter('population_estimate_2017', 'home_invasion_count')

#### United States Regions

In [ ]:
# Census regions of the United States (source: https://www2.census.gov/geo/pdfs/maps-data/maps/reference/us_regdiv.pdf)
northeast = ['Connecticut', 'Maine', 'Massachusetts', 'New Hampshire', 'Rhode Island', 'Vermont', 'New Jersey', 'New York', 'Pennsylvania'] 
midwest = ['Indiana', 'Illinois', 'Michigan', 'Ohio', 'Wisconsin', 'Iowa', 'Kansas', 'Minnesota', 'Missouri', 'Nebraska', 'North Dakota', 'South Dakota'] 
south = ['Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Maryland', 'North Carolina', 'South Carolina', 'Virginia', 'West Virginia', 'Alabama', 'Kentucky', 'Mississippi', 'Tennessee', 'Arkansas', 'Louisiana', 'Oklahoma', 'Texas'] 
west = ['Arizona', 'Colorado', 'Idaho', 'New Mexico', 'Montana', 'Utah', 'Nevada', 'Wyoming', 'Alaska', 'California', 'Hawaii', 'Oregon', 'Washington']

In [ ]:
# states in northeast region by number of incidents
df_gun_violence.loc[df_gun_violence['state'].isin(northeast)]['state'].value_counts()

In [ ]:
# states in midwest region by number of incidents
df_gun_violence.loc[df_gun_violence['state'].isin(midwest)]['state'].value_counts()

In [ ]:
# states in south region by number of incidents
df_gun_violence.loc[df_gun_violence['state'].isin(south)]['state'].value_counts()

In [ ]:
# states in west region by number of incidents
df_gun_violence.loc[df_gun_violence['state'].isin(west)]['state'].value_counts()

## Mean Income 2016 Analysis

In [ ]:
df_income.info()

In [ ]:
df_income.head()

In [ ]:
# dropped US average income and 'Notes'
df_income.drop(df_income.index[0], inplace=True)
df_income.drop(df_income.index[52:], inplace=True)

In [ ]:
df_income

In [ ]:
# renamed columns
# dropped '$' and convert from str to int
df_income = df_income.rename(index=str, columns={"Location":"state", "Median Annual Household Income": "median_annual_household_income_2016"})
df_income['median_annual_household_income_2016'] = pd.to_numeric(df_income['median_annual_household_income_2016'].str[1:], downcast='integer')
df_income

In [ ]:
# merged both dataframe(gun-violence && 2016_income)
df_gun_income = pd.merge(df_gun_violence, df_income, on="state", how="outer")
df_gun_income

In [ ]:
# merge incident count and mean income (mean annual income household 2016 only) per state
incident_count_per_state = df_gun_income['state'].value_counts().to_frame().reset_index().rename(index=str, columns={"index":"state", "state":"incident_count"})
state_incidentCount_medianIncome = pd.merge(incident_count_per_state, df_income, on="state", how="outer")
state_incidentCount_medianIncome


In [ ]:
# graphed (median_income && incident_count)
state_incidentCount_medianIncome.plot.scatter('median_annual_household_income_2016', 'incident_count')

## NRA Spending

In [ ]:
# no null values 
df_nra_spending.info()

In [ ]:
df_nra_spending.head(10)

In [ ]:
# From this dataset, we will attempt to correlate direct support, independent support, and grand total to
# number of incidents (separately)

In [ ]:
# convert monetary object of form '$d,ddd' to integer dddd
def convertToNumeric(string):
    numeric = ''
    for character in string:
        if character not in ['$', ',']:
            numeric += character
    return int(numeric)

# convert monetary values
df_nra_spending['nra_direct_support_numeric'] = df_nra_spending['NRA Direct Support'].apply(func=convertToNumeric)
df_nra_spending['nra_independent_support_numeric'] = df_nra_spending['NRA Independent Support'].apply(func=convertToNumeric)
df_nra_spending['nra_grand_total_numeric'] = df_nra_spending['NRA Grand Total'].apply(func=convertToNumeric)

# extract state code
df_nra_spending['Abbreviation'] = df_nra_spending['Distid'].str[0:2]

# extract significant columns
# Negative values for 'NRA Grand Total' mean that NRA likely opposes candidate
df_nra_spending_n = df_nra_spending.merge(df_state_codes, how='inner', left_on='Abbreviation', right_on='Abbreviation')[['First Race', 'State', 'Party', 'Office', 'nra_direct_support_numeric', 'nra_independent_support_numeric', 'nra_grand_total_numeric']].rename(index=str, columns={'First Race':'first_race', 'State':'state', 'Party':'party', 'Office':'office', 'nra_direct_support_numeric':'nra_direct_support', 'nra_independent_support_numeric':'nra_independent_support', 'nra_grand_total_numeric':'nra_grand_total'})

df_nra_spending_n.head()

#### NRA direct support (all candidates) vs. total number of incidents

In [ ]:
df_nra_direct_support_all_years = df_nra_spending_n.groupby(by='state')['nra_direct_support'].sum().reset_index()
df_incident_counts_2013_2018 = df_gun_violence['state'].value_counts().reset_index().rename(index=str, columns={'index':'state', 'state':'incident_count_2013_2018'})

df_nrads_ni = df_nra_direct_support_all_years.merge(df_incident_counts_2013_2018, how='inner', left_on='state', right_on='state')

df_nrads_ni.plot.scatter('nra_direct_support', 'incident_count_2013_2018')
df_nrads_ni.corr(method='pearson')

#### NRA independent support (all candidates) vs. total number of incidents

In [ ]:
df_nra_independent_support_all_years = df_nra_spending_n.groupby(by='state')['nra_independent_support'].sum().reset_index()

df_nrais_ni = df_nra_independent_support_all_years.merge(df_incident_counts_2013_2018, how='inner', left_on='state', right_on='state')

df_nrais_ni.plot.scatter('nra_independent_support', 'incident_count_2013_2018')
df_nrais_ni.corr(method='pearson')

#### NRA grand total (all candidates) vs. total number of incidents

In [ ]:
df_nra_grand_total_all_years = df_nra_spending_n.groupby(by='state')['nra_grand_total'].sum().reset_index()

df_nragt_ni = df_nra_grand_total_all_years.merge(df_incident_counts_2013_2018, how='inner', left_on='state', right_on='state')

df_nragt_ni.plot.scatter('nra_grand_total', 'incident_count_2013_2018')
df_nragt_ni.corr(method='pearson')

#### NRA direct support (all candidates whose first race >= 2013) vs. total number of incidents

In [ ]:
df_nra_direct_support_after_2013 = df_nra_spending_n.loc[df_nra_spending_n['first_race'] >= 2013].groupby(by='state')['nra_direct_support'].sum().reset_index()

df_nrads_ni_a2013 = df_nra_direct_support_after_2013.merge(df_incident_counts_2013_2018, how='inner', left_on='state', right_on='state')

df_nrads_ni_a2013.plot.scatter('nra_direct_support', 'incident_count_2013_2018')
df_nrads_ni_a2013.corr(method='pearson')

#### NRA independent support (all candidates whose first race >= 2013) vs. total number of incidents

In [ ]:
df_nra_independent_support_after_2013 = df_nra_spending_n.loc[df_nra_spending_n['first_race'] >= 2013].groupby(by='state')['nra_independent_support'].sum().reset_index()

df_nrais_ni_a2013 = df_nra_independent_support_after_2013.merge(df_incident_counts_2013_2018, how='inner', left_on='state', right_on='state')

df_nrais_ni_a2013.plot.scatter('nra_independent_support', 'incident_count_2013_2018')
df_nrais_ni_a2013.corr(method='pearson')

#### NRA grand total (all candidates whose first race >= 2013) vs. total number of incidents

In [ ]:
df_nra_grand_total_after_2013 = df_nra_spending_n.loc[df_nra_spending_n['first_race'] >= 2013].groupby(by='state')['nra_grand_total'].sum().reset_index()

df_nragt_ni_a2013 = df_nra_grand_total_after_2013.merge(df_incident_counts_2013_2018, how='inner', left_on='state', right_on='state')

df_nragt_ni_a2013.plot.scatter('nra_grand_total', 'incident_count_2013_2018')
df_nragt_ni_a2013.corr(method='pearson')

## Unemployment

In [ ]:
df_unemployment

In [ ]:
# Use the columns 'geography' and 'unemployment rate'
df_unemployment = df_unemployment[['Geography', 'Unemployment rate; Estimate; Population 16 years and over']]
df_unemployment

In [ ]:
# dropped United States and Puerto Rico coloumn
df_unemployment = df_unemployment.drop(df_unemployment.index[0])
df_unemployment = df_unemployment.drop(df_unemployment.index[51])
df_unemployment

In [ ]:
# renamed columns
df_unemployment = df_unemployment.rename(index=str, columns={"Geography":"state", "Unemployment rate; Estimate; Population 16 years and over": "unemployment_rate_of_2017"})
df_unemployment

In [ ]:
df_unemployment.info()

In [ ]:
# include only 2017 related accidents
# df_gun_violence_new_2017 = df_gun_violence.loc[df_gun_violence.year == 2017]
# df_gun_violence_new_2017

In [ ]:
# merged both dataframe(gun-violence && 2017_employment)
df_gun_unemployment = pd.merge(df_gun_violence, df_unemployment, on="state", how="outer")
df_gun_unemployment

In [ ]:
# merge incident count and unemployment per state
incident_count_per_state = df_gun_unemployment['state'].value_counts().to_frame().reset_index().rename(index=str, columns={"index":"state", "state":"incident_count"})
state_incidentCount_unemployment = pd.merge(incident_count_per_state, df_unemployment, on="state", how="outer")
# state_incidentCount_unemployment.to_csv('unemployment_2017.csv',index=False)

In [ ]:
# graphed (unemployment && incident_count)
state_incidentCount_unemployment.plot.scatter('unemployment_rate_of_2017', 'incident_count')

In [ ]:
state_incidentCount_unemployment.corr()['unemployment_rate_of_2017'].sort_values(ascending=False)[:]